# 데이터 전처리 부분

In [1]:
import pandas as pd

In [2]:
columns = ['id', 'text', 'label']

train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna()
test_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

In [3]:
train_data.head(2)

,id,text,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1


In [4]:
test_data.head(2)

,id,text,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0


In [5]:
train_data.to_csv('C:\\Users\\abc\\jupyter\\pytorch\\Sentiment_analysis\\train_data.csv', index = False)
test_data.to_csv('C:\\Users\\abc\\jupyter\\pytorch\\Sentiment_analysis\\test_data.csv', index = False)

In [6]:
from eunjeon import Mecab
mecab = Mecab()

In [7]:
import torch
from torchtext import data

TEXT = data.Field(tokenize=mecab.morphs)
LABEL = data.LabelField(dtype=torch.float)

In [8]:
fields = {'text':('text', TEXT), 'label':('label', LABEL)}

In [9]:
train_data, test_data = data.TabularDataset.splits(
    path = 'C:\\Users\\abc\\jupyter\\pytorch\\Sentiment_analysis',
    train = 'train_data.csv',
    test = 'test_data.csv',
    format = 'csv',
    fields = fields)

In [10]:
vars(train_data[0]), vars(train_data[1])

({'text': ['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'},
 {'text': ['흠',
   '.',
   '..',
   '포스터',
   '보고',
   '초딩',
   '영화',
   '줄',
   '.',
   '...',
   '오버',
   '연기',
   '조차',
   '가볍',
   '지',
   '않',
   '구나'],
  'label': '1'})

In [11]:
print('훈련용 데이터 수 : ', len(train_data))
print('테스트용 데이터 수 : ', len(test_data))

훈련용 데이터 수 :  149995
테스트용 데이터 수 :  49997


In [12]:
SEED = 1234

torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device == 'cuda':
    torch.cuda.manual_seed_all(SEED)

In [13]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [14]:
print('훈련용 데이터 수 : ', len(train_data))
print('테스트용 데이터 수 : ', len(test_data))
print('검증용 데이터 수 : ', len(valid_data))

훈련용 데이터 수 :  104996
테스트용 데이터 수 :  49997
검증용 데이터 수 :  44999


In [15]:
TEXT.build_vocab(train_data)
len(TEXT.vocab)

45495

In [16]:
TEXT.build_vocab(train_data, min_freq=2)
len(TEXT.vocab)

24985

In [17]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [18]:
print('TEXT 단어장의 갯수 : ', len(TEXT.vocab))
print('LABEL 단어장의 갯수 : ', len(LABEL.vocab))

TEXT 단어장의 갯수 :  25002
LABEL 단어장의 갯수 :  2


In [19]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 112312), ('이', 51529), ('는', 46855), ('영화', 40081), ('다', 38736), ('고', 32805), ('하', 31225), ('도', 23996), ('의', 23642), ('가', 23420), ('은', 21698), ('에', 21482), ('을', 20920), ('한', 17532), ('보', 17463), ('..', 16038), ('게', 15566), (',', 15390), ('들', 15047), ('!', 13782)]


In [20]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000001C3D295CD00>>, {'<unk>': 0, '<pad>': 1, '.': 2, '이': 3, '는': 4, '영화': 5, '다': 6, '고': 7, '하': 8, '도': 9, '의': 10, '가': 11, '은': 12, '에': 13, '을': 14, '한': 15, '보': 16, '..': 17, '게': 18, ',': 19, '들': 20, '!': 21, '지': 22, '를': 23, '있': 24, '없': 25, '?': 26, '좋': 27, '나': 28, '었': 29, '만': 30, '는데': 31, '너무': 32, '적': 33, '봤': 34, '안': 35, '로': 36, '정말': 37, '음': 38, '으로': 39, '것': 40, '아': 41, '네요': 42, '재밌': 43, '어': 44, '점': 45, '같': 46, '진짜': 47, '지만': 48, '했': 49, '에서': 50, '기': 51, '않': 52, '네': 53, '거': 54, '았': 55, '수': 56, '되': 57, 'ㅋㅋ': 58, '면': 59, '과': 60, '인': 61, '말': 62, '연기': 63, '최고': 64, '잘': 65, '주': 66, '~': 67, '내': 68, '평점': 69, '어요': 70, '던': 71, '이런': 72, '와': 73, '생각': 74, '할': 75, 'ㅋㅋㅋ': 76, '1': 77, '해': 78, '스토리': 79, '겠': 80, '왜': 81, '습니다': 82, '...': 83, '드라마': 84, '아니': 85, '싶': 86, '그': 87, '듯': 88, '사람': 89, '함': 90, '때': 91, '감동': 92, '더': 93, '배우': 94, '까지':

In [21]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort_key = lambda x: len(x.text),
    sort_within_batch = False)

In [22]:
next(iter(train_iterator)).text.shape

torch.Size([77, 64])

# 모델 생성 부분

In [23]:
import torch.nn as nn

In [24]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [25]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [26]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print('이 모델은 ', count_parameters(model), '개의 파라미터를 가지고 있다')

이 모델은  2592105 개의 파라미터를 가지고 있다


# 모델 훈련, 검증 함수 생성 부분

In [27]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [28]:
criterion = nn.BCEWithLogitsLoss()

In [29]:
model = model.to(device)
criterion = criterion.to(device)

In [30]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [31]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad() # 경사 초기화
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward() # 미분값 자동 갱신
        optimizer.step() # 매개변수 업데이트
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# 모델 검증 부분

In [34]:
N_EPOCHS = 5
best_valid_loss = float('inf')

In [35]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:0.2f}%')
    print(f'\t  Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:0.2f}%')

Epoch: 01 | Epoch Time: 0m 12s
	 Train Loss: 0.693 | Train Acc: 49.97%
	  Val. Loss: 0.692 |  Val. Acc: 53.13%
Epoch: 02 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.18%
	  Val. Loss: 0.691 |  Val. Acc: 53.57%
Epoch: 03 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.29%
	  Val. Loss: 0.691 |  Val. Acc: 53.70%
Epoch: 04 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.13%
	  Val. Loss: 0.691 |  Val. Acc: 53.78%
Epoch: 05 | Epoch Time: 0m 11s
	 Train Loss: 0.693 | Train Acc: 50.07%
	  Val. Loss: 0.690 |  Val. Acc: 53.83%


In [36]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.691 | Test Acc: 53.30%
